In [9]:
%%capture
!pip install -q tensorflow_text
!pip install -q simpleneighbors[annoy]
!pip install -q nltk
!pip install -q tqdm
!pip3 install tensorflow_text>=2.0.0rc0

In [10]:
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text
import seaborn as sn

In [16]:
# Test cases for how much data can be used without filling up the RAM
# 300 - OK
# 350 - OK 
# 400 - OK 
# 425 - OK 
# 450 - OK 
# 480 - FAIL

In [13]:
# data = pd.read_excel('WHO_FAQ.xlsx')
data = pd.read_csv('covidQ&A.csv')
data = data.iloc[:450]

In [11]:
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-qa/3')

In [14]:
# model
# data.iloc[1]["Answere"]
response_encoding = module.signatures['response_encoder'](
    input=tf.constant(data.Answere),
    context=tf.constant(data.Answere)
)['outputs']
# question_encoding = module.signatures['question_encoder'](
#     tf.constant(data.Context)
# )

In [15]:
def ask_question():
  question = str(input("Please ask your question? "))
  return question

In [31]:
def find_answere(question):
  if(question == ""):
    return data.iloc[0].Answere

  question = question.strip()
  question_encoding = module.signatures['question_encoder'](
      tf.constant([question])
  )['outputs']
  similarity = tf.keras.losses.cosine_similarity(response_encoding,question_encoding)
  # print(similarity)
  ans_index = np.argmin(similarity)
  # all_ans_index = np.argsort(similarity)[-3:]
  # print(data.iloc[all_ans_index[2]])
  # print(all_ans_index)
  answere = data.iloc[ans_index].Answere
  return answere


In [17]:
def main():
  print("Type quit to exit the chat")
  while True:
    question = ask_question()
    if(check_quit(question)):
      break
    answere = find_answere(question)
    print(answere)

In [18]:
def check_quit(word):
  word = word.strip().lower()
  if(word == "quit"):
    return True
  else:
    return False

In [32]:
main()

Type quit to exit the chat
Please ask your question? After covid 
Unnamed: 0                                                    0
Question      I want to change my contraceptive method – is ...
Answere       Yes. It may be difficult however, to access al...
Name: 143, dtype: object
Severe allergic reactions have occurred rarely to some of the COVID vaccines. A severe allergic reaction – such as anaphylaxis – is a potential but rare side effect with any vaccine. In persons with a known risk, such as previous experience of an allergic reaction to a previous dose of the vaccine or any of the known components in the vaccine, precautions may need to be taken.WHO recommends that healthcare providers assess the risk for severe allergic reactions prior to giving a COVID-19 vaccine by inquiring about previous reactions or known allergies to any components in the vaccine.  All immunization providers should be trained to recognize severe allergic reactions and take steps to treat such reactions i